In [37]:
import pandas as pd
import MySQLdb
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import numpy as np
import jsonify

In [38]:
db = MySQLdb.connect("69.89.14.35","360feedback","360feedback@","ttn_central_v1" )
db.autocommit=False

In [39]:
df = pd.read_sql_query(sql="""SELECT 
    360_v1_respondent_feedback.*
FROM
    ttn_central_v1.360_v1_respondent_feedback
        JOIN
    360_v1_respondent_fs_rel_mapper ON 360_v1_respondent_fs_rel_mapper.RFRM_ID = 360_v1_respondent_feedback.RFRM_ID
WHERE
    360_v1_respondent_feedback.SSUBM_ID IS NOT NULL
        AND 360_v1_respondent_fs_rel_mapper.PROJ_ID = 525;""",con=db)

In [40]:
df = df[~df.SUBJECTIVE.isin(["nopes","","."])]

In [41]:
df["sub_small"]=df.SUBJECTIVE.map(lambda x: x.lower())

In [42]:
df.head()

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID,sub_small
0,188889,None,0,77,Deliver the promise\r\nTeamwork and realtionship,6343,deliver the promise\r\nteamwork and realtionship
1,188890,None,0,78,Entrepreneurship\r\nsocial responsibility,6343,entrepreneurship\r\nsocial responsibility
2,167598,None,0,77,Learning & Deliver the promise: \r\nMr. Sharma...,6345,learning & deliver the promise: \r\nmr. sharma...
3,167599,None,0,78,Respect for the Individual & Teamwork and Rela...,6345,respect for the individual & teamwork and rela...
4,172862,None,0,77,1. Learning- Learn from all level irrespective...,6346,1. learning- learn from all level irrespective...


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
df['tokens']=df.SUBJECTIVE.apply(lambda x: nltk.wordpunct_tokenize(x))
df.tokens.head()

0    [Deliver, the, promise, Teamwork, and, realtio...
1           [Entrepreneurship, social, responsibility]
2    [Learning, &, Deliver, the, promise, :, Mr, .,...
3    [Respect, for, the, Individual, &, Teamwork, a...
4    [1, ., Learning, -, Learn, from, all, level, i...
Name: tokens, dtype: object

In [71]:
stemmer = SnowballStemmer(language="english",ignore_stopwords=False)
wordnet_lemmatizer = WordNetLemmatizer()
df['stemmed'] = df.tokens.map(lambda x: " ".join([stemmer.stem(y) for y in x if y.isalpha()]))
df['stemmedarr'] = df.tokens.map(lambda x: set([stemmer.stem(y) for y in x if y.isalpha()]))
df.head()

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID,sub_small,tokens,stemmed,stemmedarr,close,stmt_1_close,stmt_1_close_arr
0,188889,None,0,77,Deliver the promise\r\nTeamwork and realtionship,6343,deliver the promise\r\nteamwork and realtionship,"[Deliver, the, promise, Teamwork, and, realtio...",deliv the promis teamwork and realtionship,"{and, realtionship, teamwork, deliv, promis, the}",1,0,[]
1,188890,None,0,78,Entrepreneurship\r\nsocial responsibility,6343,entrepreneurship\r\nsocial responsibility,"[Entrepreneurship, social, responsibility]",entrepreneurship social respons,"{social, respons, entrepreneurship}",0,0,[]
2,167598,None,0,77,Learning & Deliver the promise: \r\nMr. Sharma...,6345,learning & deliver the promise: \r\nmr. sharma...,"[Learning, &, Deliver, the, promise, :, Mr, .,...",learn deliv the promis mr sharma is a fast lea...,"{a, learn, sharma, is, deliv, learner, ani, fa...",2,1,[Learning]
3,167599,None,0,78,Respect for the Individual & Teamwork and Rela...,6345,respect for the individual & teamwork and rela...,"[Respect, for, the, Individual, &, Teamwork, a...",respect for the individu teamwork and relation...,"{and, all, individu, maintain, good, respect, ...",1,0,[]
4,172862,None,0,77,1. Learning- Learn from all level irrespective...,6346,1. learning- learn from all level irrespective...,"[1, ., Learning, -, Learn, from, all, level, i...",learn learn from all level irrespect of hierar...,"{and, all, from, risk, execut, level, of, age,...",3,1,[Learning]


In [72]:
# Starting with the CountVectorizer/TfidfTransformer approach...
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
 
cvec = CountVectorizer(stop_words='english', min_df=1, max_df=.5, ngram_range=(1,2))
cvec

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [73]:
# Calculate all the n-grams found in all documents
from itertools import islice
cvec.fit(df.stemmed)
list(islice(cvec.vocabulary_.items(), 20))

[(u'individu bias', 55),
 (u'execut', 38),
 (u'social local', 121),
 (u'focus', 41),
 (u'inculc cost', 53),
 (u'bias maintain', 7),
 (u'collabor', 10),
 (u'collabor way', 11),
 (u'good exampl', 44),
 (u'hierarchi', 48),
 (u'high learn', 51),
 (u'superior', 125),
 (u'teamwork relationship', 135),
 (u'fast learner', 40),
 (u'learn deliv', 63),
 (u'activ', 0),
 (u'work execut', 148),
 (u'local', 71),
 (u'risk dure', 106),
 (u'good', 43)]

In [74]:
# Check how many total n-grams we have
ngramrange=len(cvec.vocabulary_)

In [75]:
# Check how many total n-grams we have
len(cvec.vocabulary_)

149

In [76]:
cvec_counts = cvec.transform(df.stemmed)
print 'sparse matrix shape:', cvec_counts.shape
print 'nonzero count:', cvec_counts.nnz
print 'sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1]))

SyntaxError: invalid syntax (<ipython-input-76-55ca98c76fda>, line 2)

In [77]:
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(20)

NameError: name 'cvec_counts' is not defined

In [78]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

NameError: name 'cvec_counts' is not defined

In [79]:
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

NameError: name 'transformed_weights' is not defined

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(min_df=.0025, max_df=.1, stop_words='english')
tvec_weights = tvec.fit_transform(df.stemmed.dropna())
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(10)

,term,weight
35,realtionship,0.100000
12,deliveri,0.100000
31,need,0.060302
44,support,0.050000
30,manag,0.050000
29,maintain,0.050000
27,local,0.050000
17,fast,0.050000
42,stakehold,0.050000
43,superior,0.050000


In [81]:
weights_df.size

100

In [82]:
top_weight=weights_df.sort_values(by='weight', ascending=False).head(50)
top_weight.head(5)

,term,weight
35,realtionship,0.100000
12,deliveri,0.100000
31,need,0.060302
44,support,0.050000
30,manag,0.050000


In [83]:
df["close"]=df.stemmedarr.map(lambda x: len([y for y in x if y in top_weight.term.values]))

In [84]:
df.sort_values(by="close",ascending=False).head()

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID,sub_small,tokens,stemmed,stemmedarr,close,stmt_1_close,stmt_1_close_arr
6,168679,None,0,77,He has very high learning grasp and seen in va...,6347,he has very high learning grasp and seen in va...,"[He, has, very, high, learning, grasp, and, se...",he has veri high learn grasp and seen in vario...,"{and, high, in, seen, shown, teamwork, way, va...",10,0,[]
8,202921,None,0,77,Teamwork and Relationships - Excellent rapport...,6349,teamwork and relationships - excellent rapport...,"[Teamwork, and, Relationships, -, Excellent, r...",teamwork and relationship excel rapport with u...,"{and, all, love, individu, attitud, respect, r...",10,0,[]
4,172862,None,0,77,1. Learning- Learn from all level irrespective...,6346,1. learning- learn from all level irrespective...,"[1, ., Learning, -, Learn, from, all, level, i...",learn learn from all level irrespect of hierar...,"{and, all, from, risk, execut, level, of, age,...",8,1,[Learning]
9,202922,None,0,78,Entrepreneurship - Needs to inculcate cost con...,6349,entrepreneurship - needs to inculcate cost con...,"[Entrepreneurship, -, Needs, to, inculcate, co...",entrepreneurship need to inculc cost conscious...,"{and, on, conscious, inculc, self, social, act...",8,0,[]
2,167598,None,0,77,Learning & Deliver the promise: \r\nMr. Sharma...,6345,learning & deliver the promise: \r\nmr. sharma...,"[Learning, &, Deliver, the, promise, :, Mr, .,...",learn deliv the promis mr sharma is a fast lea...,"{a, learn, sharma, is, deliv, learner, ani, fa...",4,1,[Learning]


In [85]:
[y for y in df.SUBJECTIVE.values]

['Deliver the promise\r\nTeamwork and realtionship',
 'Entrepreneurship\r\nsocial responsibility',
 'Learning & Deliver the promise: \r\nMr. Sharma is a fast learner manager, he can also deliver the promise if any.',
 'Respect for the Individual & Teamwork and Relationship: \r\nMr. Sharma can demonstrate these values to respect to the individual without biasness and he should maintain good relationship with his all colleagues.',
 '1. Learning- Learn from all level irrespective of hierarchy and age etc. 2. Entrepreneurship- take calculated risk during work execution.',
 '1. Delivery of Promise 2. Respect for individual',
 'He has very high learning grasp and seen in various situations. \r\nHe has worked in a collaborative way with team and shown good example of teamwork.',
 'He should be more social to local stakeholders and take entrepreneurship with the support of superiors.',
 'Teamwork and Relationships - Excellent rapport with UK team and also Delhi team\r\nRespect for Individual -

In [86]:
stmts = pd.read_csv("/home/ranjan/git/python-practice/jupyter/searcg_stmt.csv")

In [87]:
stmts

,STATEMENT
0,Learning & Inner Excellence
1,Respect for Individual
2,Teamwork & Relationships
3,Deliver the Promise


In [88]:
stmts['tokens']=stmts.STATEMENT.apply(lambda x: nltk.wordpunct_tokenize(x))
stmts['stemmedarr'] = stmts.tokens.map(lambda x: set([stemmer.stem(y) for y in x if y.isalpha()]))
stmts.head()

,STATEMENT,tokens,stemmedarr
0,Learning & Inner Excellence,"[Learning, &, Inner, Excellence]","{excel, inner, learn}"
1,Respect for Individual,"[Respect, for, Individual]","{respect, individu, for}"
2,Teamwork & Relationships,"[Teamwork, &, Relationships]","{teamwork, relationship}"
3,Deliver the Promise,"[Deliver, the, Promise]","{the, promis, deliv}"


In [89]:
stmts.loc[[2]]

,STATEMENT,tokens,stemmedarr
2,Teamwork & Relationships,"[Teamwork, &, Relationships]","{teamwork, relationship}"


In [90]:
stmts.loc[[1]].stemmedarr.values[0]

{u'for', u'individu', u'respect'}

In [91]:
df["stmt_1_close"]=df.stemmedarr.map(lambda x: len([y for y in x if y in stmts.loc[[0]].stemmedarr.values[0]]))
df["stmt_1_close_arr"]=df.stemmedarr.map(lambda x: [y for y in x if y in stmts.loc[[0]].stemmedarr.values[0]])

In [92]:
df.head()

,RF_ID,CSM_ID,RATE,SSUBM_ID,SUBJECTIVE,RFRM_ID,sub_small,tokens,stemmed,stemmedarr,close,stmt_1_close,stmt_1_close_arr
0,188889,None,0,77,Deliver the promise\r\nTeamwork and realtionship,6343,deliver the promise\r\nteamwork and realtionship,"[Deliver, the, promise, Teamwork, and, realtio...",deliv the promis teamwork and realtionship,"{and, realtionship, teamwork, deliv, promis, the}",1,0,[]
1,188890,None,0,78,Entrepreneurship\r\nsocial responsibility,6343,entrepreneurship\r\nsocial responsibility,"[Entrepreneurship, social, responsibility]",entrepreneurship social respons,"{social, respons, entrepreneurship}",0,0,[]
2,167598,None,0,77,Learning & Deliver the promise: \r\nMr. Sharma...,6345,learning & deliver the promise: \r\nmr. sharma...,"[Learning, &, Deliver, the, promise, :, Mr, .,...",learn deliv the promis mr sharma is a fast lea...,"{a, learn, sharma, is, deliv, learner, ani, fa...",4,1,[learn]
3,167599,None,0,78,Respect for the Individual & Teamwork and Rela...,6345,respect for the individual & teamwork and rela...,"[Respect, for, the, Individual, &, Teamwork, a...",respect for the individu teamwork and relation...,"{and, all, individu, maintain, good, respect, ...",4,0,[]
4,172862,None,0,77,1. Learning- Learn from all level irrespective...,6346,1. learning- learn from all level irrespective...,"[1, ., Learning, -, Learn, from, all, level, i...",learn learn from all level irrespect of hierar...,"{and, all, from, risk, execut, level, of, age,...",8,1,[learn]


In [93]:
df.stemmedarr.map(lambda x: [y for y in x if y in stmts.loc[[0]].stemmedarr.values[0]])

0         []
1         []
2    [learn]
3         []
4    [learn]
5         []
6    [learn]
7         []
8    [excel]
9         []
Name: stemmedarr, dtype: object

In [94]:
dir(nltk.stem)

['ISRIStemmer',
 'LancasterStemmer',
 'PorterStemmer',
 'RSLPStemmer',
 'RegexpStemmer',
 'SnowballStemmer',
 'StemmerI',
 'WordNetLemmatizer',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__path__',
 'api',
 'isri',
 'lancaster',
 'porter',
 'regexp',
 'rslp',
 'snowball',
 'util',
 'wordnet']